In [1]:
#MDM on Total Dead Kilometers Matrix
#import timeit
#start = timeit.default_timer()

# All the program statements

import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
df_demand = {}
random.seed(21) # Use the same random seed
for i in range(0,50):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(50):
    df_demand[i] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v

df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0

np.random.seed(21) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,60, size=(sum,50))).astype(float)

random.seed(21) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(21) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(21) # Use the same random seed
doc = {}
for i in range(0,50):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom

df_cost = df_tdkom.copy(deep=True)

In [2]:
df_result = pd.DataFrame(np.zeros(df_tdkom.shape))

In [3]:
while(list(df_supply.values()) and list(df_demand.values())): 
    # To find the two lowest values column-wise
    if len(df_tdkom.index)!=1:
        import heapq
        smallest = df_tdkom.apply(lambda x: heapq.nsmallest(2, x), axis=0)
        smallest_df = pd.DataFrame(smallest.values.tolist())
        dfn = smallest_df.loc[1] - smallest_df.loc[0]
        df_pen = list(dfn)
    else:
        df_pen = df_tdkom.apply(lambda x: x.sort_values().unique()[0], axis=0)

    df_penc = {}
    for i in range(len(df_tdkom.columns)):
        df_penc[df_tdkom.columns[i]] = df_pen[i]


    max_d = 0
    for (k,v) in df_demand.items():
        if (v > max_d):
            max_d = v

    kmx = [key for key,val in df_demand.items() if val == max_d] #keys with maximum demand value


    # To find the column with maximum penalty cost, or least cost cell (maxi = that column) 
    if (len(kmx)!=1):
        maxi = 0
        for i in range(len(kmx)):
            if (df_penc[kmx[i]] > maxi):
                maxi = df_penc[kmx[i]]


        kmpen = [key for key,val in df_penc.items() if val == maxi] #keys with maximum penalty cost

        # To find the cell with smallest unit cost in case of a VAM tie
        if (len(kmpen)!=1):
            minval = []
            for i in kmpen:
                minval.append(df_tdkom[i].min())
            p = min(minval)


            for i in kmpen:
                for j in df_tdkom.index:
                    if (p == df_tdkom.loc[j,i]):
                        max_key = i #To store max_key
        else:
            max_key = kmpen[0]
    
    elif (len(kmx) == 1):
        max_key = kmx[0] 

    lst = []
    for j in df_tdkom.columns:
        if (j == max_key): # For same values of demand, with different penalty cost
            #print(j)
            while (df_demand[j] != 0):
                df_nc_ = df_tdkom.min(axis=0)[j]
                #print(df_nc_)
                for i in df_tdkom.index:
                    if(df_tdkom.loc[i,j] == df_nc_):
                        lst.append([i,j])     
                        df_result.loc[lst[0][0],lst[0][1]] = min(df_supply[lst[0][0]],df_demand[lst[0][1]])
                        df_supply[lst[0][0]] = df_supply[lst[0][0]] - df_result.loc[lst[0][0],lst[0][1]]
                        df_demand[lst[0][1]] = df_demand[lst[0][1]] - df_result.loc[lst[0][0],lst[0][1]]
                        #print(df_result)
                        lst = []

                    if (df_supply[i] == 0):
                        df_tdkom = df_tdkom.drop(i, axis = 0)
                        del df_supply[i]

            if (df_demand[j] == 0):
                df_tdkom = df_tdkom.drop(j, axis=1)
                del df_demand[j]

In [4]:
df_result

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_final = df_cost.mul(df_result, axis = 0)
df_final.to_numpy().sum()

732896.224805628

stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))